<a href="https://colab.research.google.com/github/jonathantlima/DataTalksClub/blob/main/Homework%2301.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Homework #01**

###`The goal of this homework is to train a simple model for predicting the duration of a ride - similar to what we did in this module.`

This notebook displays the solution to homework #01 from the 2024 version of the MLOps-Zoomcamp course, provided by DataTalks.Club

Download dataset in this [link](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page).

In [1]:
# loading batteries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn.metrics import accuracy_score, mean_squared_error

In [2]:
# Linking to GDrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
pip install pyarrow

In [4]:
# Sets the path for the corresponding file "yellow_tripdata_2023-01.parquet"
yellow_cab_jan_23 = "/content/drive/MyDrive/DataTalks.Club/MLOps Zoomcamp 2024/Homework #01/yellow_tripdata_2023-01.parquet"

In [5]:
# Loads the file as Pandas DataFrame
yellow_cab = pd.read_parquet(yellow_cab_jan_23)

In [6]:
# Checking if the DataFrame has been correctly generated
yellow_cab.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


## **Q1**: Read the data for January. How many columns are there?

In [7]:
# Shape attribute shows how many rows and columns there are in a DataFrame
yellow_cab.shape

(3066766, 19)

As a result one can check this DataFrame is encompassed by 19 columns.

## **Q2**: What's the standard deviation of the trips duration in January?

Duration can be calculated as the time difference between `tpep_pickup_datetime` and `tpep_dropoff_datetime`

In [8]:
# Checking 'tpep_pickup_datetime' types
type(yellow_cab['tpep_pickup_datetime'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [9]:
# Checking 'tpep_dropoff_datetime' types
type(yellow_cab['tpep_dropoff_datetime'][0])

pandas._libs.tslibs.timestamps.Timestamp

The above entries are both datetime data with timestamp. Then, 'Duration' can readily calculated in a new column.

In [10]:
yellow_cab['Duration'] = yellow_cab['tpep_dropoff_datetime'] - yellow_cab['tpep_pickup_datetime']
yellow_cab['Duration'].head()

0   0 days 00:08:26
1   0 days 00:06:19
2   0 days 00:12:45
3   0 days 00:09:37
4   0 days 00:10:50
Name: Duration, dtype: timedelta64[us]

In [11]:
# Checking for NaN
yellow_cab['Duration'].isna().sum()

0

In [12]:
yellow_cab['Duration (min)'] = yellow_cab['Duration'].dt.total_seconds()/60

In [13]:
# Standard deviation for duration
yellow_cab['Duration (min)'].std()

42.59435124195458

Thus, the standar deviation for duration, in minutes, equals 42.59

## **Q3**: What fraction of the records left after you dropped the outliers?

In [14]:
outliers_free = yellow_cab.loc[(yellow_cab['Duration (min)'] >= 1) & (yellow_cab['Duration (min)'] <= 60)]

In [15]:
outliers_free.shape

(3009173, 21)

In [16]:
outliers_free.shape[0] / yellow_cab.shape[0]

0.9812202822125979

Then, after removing the outliers there is a remaining fraction of data corresponding to 98% of the original data.

## **Q4**: one-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

*   Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
*   Fit a dictionary vectorizer
*   Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [17]:
yellow_copy = outliers_free.copy()

In [18]:
yellow_copy.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee', 'Duration',
       'Duration (min)'],
      dtype='object')

In [19]:
yellow_copy = yellow_copy[['PULocationID','DOLocationID']]

In [20]:
yellow_copy.head()

,PULocationID,DOLocationID
0,161,141
1,43,237
2,48,238
3,138,7
4,107,79


In [21]:
# Checking for missing values
yellow_copy.isna().sum()

PULocationID    0
DOLocationID    0
dtype: int64

In [22]:
len(yellow_copy.PULocationID.unique())

255

In [23]:
len(yellow_copy.DOLocationID.unique())

260

In [24]:
yellow_copy = yellow_copy.astype('str')

In [25]:
train_dicts = yellow_copy.to_dict(orient='records')

In [26]:
dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

In [27]:
X_train.shape

(3009173, 515)

In [28]:
len(dv.feature_names_)

515

## **Q5**: train the model

Now let's use the feature matrix from the previous step to train a model.

*   Train a plain linear regression model with default parameters
*   Calculate the RMSE of the model on the training data

What's the RMSE on train?

In [29]:
y_train = outliers_free['Duration (min)'].values

In [30]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [31]:
y_pred = lr.predict(X_train)

In [32]:
mse = mean_squared_error(y_train, y_pred, squared=False)
print(f'Mean squared error: {mse}')

Mean squared error: 7.649261929771859


## **Q6**: Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [82]:
february = pd.read_parquet("/content/drive/MyDrive/DataTalks.Club/MLOps Zoomcamp 2024/Homework #01/yellow_tripdata_2023-02.parquet")

In [83]:
february.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [84]:
february.shape

(2913955, 19)

In [85]:
february[['PULocationID','DOLocationID']].isna().sum()

PULocationID    0
DOLocationID    0
dtype: int64

In [86]:
february['Duration'] = (february['tpep_dropoff_datetime'] - february['tpep_pickup_datetime']).dt.total_seconds()/60

In [87]:
february.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,Duration
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00,1.683333
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00,0.233333
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00,0.233333
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25,32.083333
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00,13.300000


In [96]:
february = february[(february.Duration >= 1) & (february.Duration <= 60)].copy()

In [97]:
categorical = ['PULocationID','DOLocationID']
df_val = february[categorical].astype('str')

In [98]:
df_val.head()

,PULocationID,DOLocationID
0,142,163
3,132,26
4,161,145
5,148,236
6,137,244


In [99]:
val_dicts = df_val.to_dict(orient='records')

In [100]:
X_val = dv.transform(val_dicts)

In [101]:
y_test = february['Duration']

In [102]:
y_val = lr.predict(X_val)

In [103]:
mse_test = mean_squared_error(y_test, y_val, squared=False)
print(f'Mean squared error in test: {mse_test}')

Mean squared error in test: 7.811818933419717
